In [1]:
import torch, platform

print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("Built with CUDA:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

Python: 3.10.18
Torch: 2.8.0+cu129
Built with CUDA: 12.9
CUDA available: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-3B-Instruct-bnb-4bit",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth",
)

c:\Users\namth\anaconda3\envs\lung_lora\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W1008 23:53:02.122000 7512 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.11: Fast Qwen2_5_Vl patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GeForce RTX 4050 Laptop GPU. Num GPUs = 1. Max memory: 5.997 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu129. CUDA: 8.9. CUDA Toolkit: 12.9. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


งานของคุณคือ VLM แบบ สองงานพร้อมกัน: (1) จัดหมวดโรคจากภาพ X-ray (classification) + (2) สร้างคำอธิบายภาพ (captioning) บนชุด 6,000 คู่ภาพ–ข้อความ

อาการ “ทายโรคเดิมซ้ำ (เช่น normal) จนแม่นยำตกเหลือ ~0.17” มักมาจาก catastrophic forgetting / class collapse และ อิมบาลานซ์ของสัญญาณเรียนรู้ (ส่วนภาษา/แคปชันกลบงานจัดหมวด)

เป้าหมายแพทย์รังสี = ให้ “สายภาพ” เรียนรู้เด่นกว่า “สายภาษา” เพื่อไม่ให้โมเดลลื่นไปตอบคำอธิบายสวยๆ แต่ ไม่จับพยาธิสภาพ

ดังนั้นให้ “เริ่มแบบอนุรักษ์นิยม”: ฝึกเฉพาะ vision + cross-attention ก่อน, คุม LoRA ขนาดกลาง, ใส่ dropout ป้องกัน overfit, เปิด rsLoRA ให้เรียนรู้เสถียรขึ้น โดยเฉพาะกับโมเดลใหญ่/ข้อมูลไม่มาก (6k).

In [3]:
# ==========================
# 1) PEFT / LoRA config (ปรับตามโจทย์ 2 งาน: cls + cap)
#    - เปิดทั้ง vision + attention เพื่อเรียนรู้สัมพันธ์ภาพ-ข้อความ
#    - รอบแรก "กัน drift ภาษา" ด้วยการปิด language MLP; จะค่อย ๆ เปิดในรอบถัดไปได้
#    - ใช้ rsLoRA เพื่อเสถียรกับ rank สูง
# ==========================
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,    # ✅ ให้สายภาพเรียนรู้พยาธิสภาพ/feature เฉพาะทาง
    finetune_language_layers   = False,   # ✅ รอบแรกกัน drift ภาษา (ยัง generate ได้ เพราะ head/weights ถูกใช้ตอน infer)
    finetune_attention_modules = True,    # ✅ สำคัญมากสำหรับ cross-modal alignment (ภาพ↔ข้อความ)
    finetune_mlp_modules       = False,   # ✅ ลดพารามิเตอร์ช่วงเริ่ม กัน overfit / catastrophic forgetting

    r = 16,                # ✅ เริ่มที่ 16; ถ้า underfit ค่อยขยับ 32 พร้อม rsLoRA
    lora_alpha = 16,       # ✅ อย่างน้อยเท่ากับ r ตามคำแนะนำ Unsloth
    lora_dropout = 0.15,   # ✅ ช่วยกัน overfit/โหมดเดิม (เช่นตอบ "Normal" รัว ๆ)
    bias = "none",
    random_state = 3407,

    use_rslora = True,     # ✅ เสถียรกับ rank สูง/งานยาก
    loftq_config = None,   # ✅ ถ้าเทรน 4-bit แนะนำเปิด LoftQ โปรไฟล์ C เพื่อคุณภาพ
    # target_modules = ["q_proj","k_proj","v_proj","o_proj"],  # ↔ ถ้าจะเริ่มเฉพาะ attention เท่านั้น ค่อยปลดคอมเมนต์
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.15.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


In [4]:
from datasets import load_from_disk

hf = load_from_disk("lung8_image_text")

# แบ่ง train/test (30%) ก่อน
splits = hf.train_test_split(test_size=0.3, seed=42, shuffle=True)
train_hf = splits["train"]
tmp_hf   = splits["test"]

# แบ่ง tmp ให้เป็น val/test อย่างละครึ่ง → ได้ 15/15
vt = tmp_hf.train_test_split(test_size=0.5, seed=42, shuffle=True)
val_hf  = vt["train"]
test_hf = vt["test"]

In [5]:
train_hf

Dataset({
    features: ['image', 'text', '__class__'],
    num_rows: 4259
})

In [6]:
instruction = "Describe the chest X-ray using precise clinical terms. Identify one main diagnostic category from: Chest_Changes, Degenerative_Infectious, Higher_Density, Inflammatory_Pneumonia, Lower_Density, Mediastinal_Changes, Normal, or Obstructive."

def convert_to_conversation(sample):
    cls_name = sample["__class__"]
    description = sample["text"]

    answer = f"Class: {cls_name}\nExplanation: {description}"
    
    conversation = [
        {"role": "user", "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image"]} ]
        },
        {"role" : "assistant", "content" : [
            {"type" : "text", "text" : answer} ]
        },
    ]
    
    return {"messages" : conversation}

train_ds = [convert_to_conversation(sample) for sample in train_hf]
val_ds = [convert_to_conversation(sample) for sample in val_hf]

In [7]:
train_ds[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Describe the chest X-ray using precise clinical terms. Identify one main diagnostic category from: Chest_Changes, Degenerative_Infectious, Higher_Density, Inflammatory_Pneumonia, Lower_Density, Mediastinal_Changes, Normal, or Obstructive.'},
    {'type': 'image',
     'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=450x450>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'Class: Lower_Density\nExplanation: Collapsed lung / visceral pleural line observed showing visceral pleural line with absent peripheral lung markings on the left side. A small amount of apical collapse is noted. The remaining lung fields are clear. No associated pleural effusion is present. Findings are compatible with Lower density (pneumothorax, pneumomediastinum, pneumoperitoneum).'}]}]}

ค่า SFTTrainer ที่เหมาะสมต้อง “กันคลาสล่ม/ตอบซ้ำ normal” และ “คุมไม่ให้ captioning กลบงานจัดหมวด” โดยใช้แบตช์เอฟเฟกทีฟใหญ่พอ, อัตราเรียนรู้พอดี, ประเมินถี่, เซฟโมเดลที่ดีที่สุด และ (ถ้าทำได้) ถ่วงน้ำหนัก loss ให้ฝั่ง classification มากกว่า

สำหรับ compute_metrics ไม่ “บังคับต้องมี” แต่ จำเป็นในทางปฏิบัติ เพื่อจับอาการคลาสล่ม/เดาทางเดียว (เช่น normal รัว ๆ) ด้วยสถิติที่สื่อความหมาย (accuracy/F1 ต่อคลาส, BLEU/CIDEr สำหรับแคปชัน) แทนจะดู loss อย่างเดียว 

In [8]:
from unsloth import is_bf16_supported, FastVisionModel
from unsloth.trainer import UnslothVisionDataCollator
from transformers import EarlyStoppingCallback, TrainerCallback
from trl import SFTTrainer, SFTConfig
import torch, math, random
import numpy as np
from collections import Counter
from typing import List, Dict, Any
import re
import unicodedata

In [9]:
# ==========================
# 2) ค่าตายตัวของงาน (label set) และตัวช่วยเล็ก ๆ
# ==========================
CLASS_LABELS = [
    "Chest_Changes", "Degenerative_Infectious", "Higher_Density",
    "Inflammatory_Pneumonia", "Lower_Density", "Mediastinal_Changes",
    "Normal", "Obstructive",
]
LABEL_SET = set(CLASS_LABELS)
LABEL_TO_ID = {c:i for i,c in enumerate(CLASS_LABELS)}

def extract_pred_class(text: str) -> str:
    """
    ดึงคลาสจากข้อความยาวของโมเดล:
    รับมือกรณีมีช่องว่าง/เคส/บรรทัดสลับ/มีเครื่องหมายพิเศษ
    """
    # หา "Class: <ชื่อคลาส>"
    m = re.search(r"(?i)class\s*:\s*([A-Za-z0-9_\- ]+)", text)
    if not m:
        return None
    raw = m.group(1).strip()
    # ทำ normalization เบื้องต้น
    cand = raw.replace(" ", "_")
    # เช็ค map เข้าคลาสที่รู้จัก (แบบหยวนๆ)
    # ลองตรงตัวก่อน
    if cand in LABEL_TO_ID: 
        return cand
    # ลองเปรียบเทียบแบบ lower-case
    for c in CLASS_LABELS:
        if cand.lower() == c.lower():
            return c
    # ถ้าหาไม่เจอให้คืน None เพื่อให้จัดเป็นผิดพลาด
    return None


In [10]:
test_pred = """
Class: mEdiAsTinal cHanges
Explanation: The chest X-ray shows increased transparency adjacent to both right and left cardiophrenic angles with variable clarity indicating partial atelectasis or pneumonia. This imaging feature suggests inflammatory change that is consistent with an underlying infectious etiology in this setting. The clinical scenario strongly supports infection leading to lung parenchymal changes. There are no visible effusions, mass lesions, or linear changes typical for entities like neoplasms or autoimmune conditions. The features are compatible with Degenerative_Infectious based on clinical and radiographic correlation, particularly considering the known clinical presentations associated with specific conditions in similar
"""

In [11]:
extract_pred_class(test_pred)

'Mediastinal_Changes'

In [12]:
def rouge_l_f1(pred: str, ref: str) -> float:
    """
    ROUGE-L F1 แบบเรียบง่าย (ไม่พึ่ง external lib) ในช่วง [0,1]
    ใช้เป็น proxy ของคุณภาพ captioning เมื่อยังไม่มี CIDEr
    """
    # แปลงเป็น token ระดับคำแบบหยาบ ๆ
    def tok(s): 
        return [w for w in s.strip().split() if w]
    x, y = tok(pred.lower()), tok(ref.lower())
    if not x or not y:
        return 0.0
    # LCS length (dynamic programming)
    m, n = len(x), len(y)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if x[i] == y[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    lcs = dp[m][n]
    prec = lcs / max(1, m)
    rec  = lcs / max(1, n)
    if prec + rec == 0:
        return 0.0
    return 2 * prec * rec / (prec + rec)


In [13]:
ref_t = """
Class: Mediastinal_Changes
Explanation: Findings are compatible with Arteriovenous malformations (pulmonary AVM may project as nodular opacity), characterized by well-circumscribed nodular opacity with suspected vascular connections may reflect pulmonary AVM (confirm on CT/angio). The nodular opacity warrants careful evaluation. Further imaging, such as a CT angiogram, is crucial for confirmation and characterization of feeding/draining vessels.
"""

ref_f = """
Class: Degenerative_Infectious
Explanation: Reticulonodular pattern / interstitial fibrosis with coarse reticular opacities with volume loss, basilar and peripheral predominance (fibrotic pattern on CXR). Findings are compatible with Pulmonary fibrosis (e.g., IPF pattern on CXR). The reticular opacities are most evident in the lower lobes and periphery. There is associated volume loss and architectural distortion. These findings are highly suggestive of a fibrotic lung disease.
"""

In [14]:
rouge_l_f1(test_pred, ref_t)

0.09333333333333334

In [15]:
rouge_l_f1(test_pred, ref_f)

0.125

In [16]:
def macro_f1_from_predictions(y_true: List[int], y_pred: List[int]) -> float:
    """
    macro-F1 แบบไม่ต้องพึ่ง sklearn (เพื่อความ self-contained)
    """
    num_classes = len(CLASS_LABELS)
    # สร้าง confusion แบบนับ TP/FP/FN ต่อคลาส
    tp = [0]*num_classes
    fp = [0]*num_classes
    fn = [0]*num_classes
    for yt, yp in zip(y_true, y_pred):
        if yp == yt:
            tp[yt] += 1
        else:
            fp[yp] += 1
            fn[yt] += 1
    f1s = []
    for c in range(num_classes):
        p = tp[c] / max(1, (tp[c] + fp[c]))
        r = tp[c] / max(1, (tp[c] + fn[c]))
        if p + r == 0:
            f1s.append(0.0)
        else:
            f1s.append(2*p*r/(p+r))
    return float(np.mean(f1s))

In [ ]:
# # ==========================
# # 3) Callback: ประเมิน cls + cap แล้ว "ล็อก" metric
# #    - ใช้ subset เพื่อเร็วและเสถียร
# #    - สร้างข้อความ prompt ตาม chat template ที่คุณใช้อยู่
# # ==========================
# class CaptionEvalCallback(TrainerCallback):
#     def __init__(self, eval_dataset, tokenizer, sample_size=256, max_new_tokens=96, seed=42):
#         self.eval_dataset = eval_dataset
#         self.tokenizer = tokenizer
#         self.sample_size = sample_size
#         self.max_new_tokens = max_new_tokens
#         self.rng = random.Random(seed)

#     def build_messages(self, image_obj) -> Dict[str, Any]:
#         """
#         แปลงเป็นรูปแบบ messages ที่สอดคล้องกับตัวอย่างของคุณ:
#         {'messages': [
#             {'role': 'user', 'content': [{'type':'text',...}, {'type':'image',...}]}
#         ]}
#         หมายเหตุ: ที่นี่เราตั้ง prompt โครงเดียวกับ data ที่สั่งบังคับ convert_to_conversation ไว้
#         """
#         return {
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": [
#                         {"type": "text",
#                             "text": "Describe the chest X-ray using precise clinical terms. "
#                                     "Identify one main diagnostic category from: Chest_Changes, Degenerative_Infectious, "
#                                     "Higher_Density, Inflammatory_Pneumonia, Lower_Density, Mediastinal_Changes, Normal, or Obstructive."},
#                         {"type": "image", "image": image_obj},
#                     ],
#                 }
#             ]
#         }

#     @torch.no_grad()
#     def on_evaluate(self, args, state, control, model=None, **kwargs):
#         model.eval()

#         # ----- สุ่ม subset จาก val_ds เพื่อลดเวลา evaluate -----
#         n = len(self.eval_dataset)
#         idxs = list(range(n))
#         self.rng.shuffle(idxs)
#         idxs = idxs[:min(self.sample_size, n)]

#         pred_classes, true_classes = [], []
#         rouge_ls = []

#         for i in idxs:
#             row = self.eval_dataset[i]
#             image_obj = row["image"]      # PIL image
#             true_cls  = row["__class__"]  # ground-truth label string

#             # ----- เตรียมอินพุตตาม template ของรุ่น -----
#             messages = self.build_messages(image_obj)
#             # Unsloth: ใช้ tokenizer.apply_chat_template ให้เป็นรูปแบบที่ model เข้าใจ
#             inputs = self.tokenizer.apply_chat_template(
#                 messages["messages"],
#                 add_generation_prompt=True,
#                 tokenize=True,
#                 return_tensors="pt"
#             ).to(model.device)

#             # ----- แนบภาพ (FastVisionModel รองรับผ่าน processor ภายในโมเดล) -----
#             # หมายเหตุ: สำหรับ Unsoth FastVisionModel ปกติ data_collator จะดูแลภาพ
#             # ที่นี่เราสาธิตแนวคิดด้วยการส่งเฉพาะ ids (ในงานจริงคุณอาจต้องใช้ processor)
#             # หากรุ่นต้องใช้ processor เฉพาะ ให้ประกอบเป็น kwargs = processor(...)

#             # ----- Generate -----
#             out = model.generate(
#                 input_ids=inputs,
#                 max_new_tokens=self.max_new_tokens,
#                 do_sample=False
#             )
#             text = self.tokenizer.decode(out[0], skip_special_tokens=True)

#             # ----- แยก "คลาส" ที่โมเดลทำนาย พร้อมคำนวณ ROUGE-L caption -----
#             pred_cls_str = extract_pred_class(text) or ""   # string ของคลาสที่พยากรณ์
#             if pred_cls_str in LABEL_SET:
#                 pred_classes.append(LABEL_TO_ID[pred_cls_str])
#             else:
#                 # ถ้าอ่านไม่ออกให้ลง -1 แล้วไปกรองออก
#                 pred_classes.append(-1)

#             true_classes.append(LABEL_TO_ID.get(true_cls, -1))

#             # เทียบ caption กับ reference
#             # - ในชุดของคุณ reference caption อยู่ใน field "text" (จากตัวอย่าง convert_to_conversation)
#             ref_caption = row.get("text", "") or ""
#             rouge_ls.append(rouge_l_f1(text, ref_caption))

#         # ----- ทำความสะอาดกรณีมี -1 -----
#         y_true_clean, y_pred_clean = [], []
#         for yt, yp in zip(true_classes, pred_classes):
#             if yt >= 0 and yp >= 0:
#                 y_true_clean.append(yt)
#                 y_pred_clean.append(yp)

#         macro_f1 = macro_f1_from_predictions(y_true_clean, y_pred_clean) if y_true_clean else 0.0
#         rougeL   = float(np.mean(rouge_ls)) if rouge_ls else 0.0

#         # ----- บันทึกค่า metric เข้าสู่ระบบของ Trainer -----
#         # ใส่ใน state.log_history ผ่าน trainer.log โดยใช้ control เปลี่ยนค่า
#         # ที่นี่เราจะใส่ลงใน kwargs['metrics'] ถ้ามี (HF >= 4.41 มักส่ง metrics เข้ามา)
#         metrics = kwargs.get("metrics", {})
#         metrics["macro_f1"] = macro_f1            # ✅ metric หลักสำหรับเลือก best
#         metrics["rougeL"]   = rougeL              # ✅ proxy ของ caption คุณภาพ (0–1)
#         # ถ้าคุณมี CIDEr จริง ให้แทนด้วย metrics["cider"] = compute_cider(...)
#         kwargs["metrics"] = metrics

#         # หมายเหตุ: ค่าใน metrics จะถูก logger ของ Trainer เก็บ และใช้ร่วมกับ EarlyStopping/metric_for_best_model
#         return control

In [28]:
# ==========================
# 3) Callback: ประเมิน cls + cap แล้ว "ล็อก" metric (เวอร์ชันทนทาน KeyError)
# ==========================
class CaptionEvalCallback(TrainerCallback):
    def __init__(self, eval_dataset, tokenizer, sample_size=256, max_new_tokens=96, seed=42):
        self.eval_dataset = eval_dataset
        self.tokenizer = tokenizer
        self.sample_size = sample_size
        self.max_new_tokens = max_new_tokens
        self.rng = random.Random(seed)

    # --- ใช้เฉพาะกรณีไม่มี messages ใน row ---
    def build_messages(self, image_obj) -> Dict[str, Any]:
        """
        กรณีชุดข้อมูลเป็นรูปแบบเก่า (มี 'image' และ 'text') แต่ยังไม่มี 'messages'
        ให้สร้างข้อความ user ตาม prompt เดิม + แนบภาพ
        """
        return {
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text",
                         "text": (
                            "Describe the chest X-ray using precise clinical terms. "
                            "Identify one main diagnostic category from: "
                            "Chest_Changes, Degenerative_Infectious, Higher_Density, "
                            "Inflammatory_Pneumonia, Lower_Density, Mediastinal_Changes, "
                            "Normal, or Obstructive."
                         )},
                        {"type": "image", "image": image_obj},
                    ],
                }
            ]
        }

    def _extract_from_row(self, row: Dict[str, Any]):
        """
        ดึงข้อมูลให้ครอบคลุมทุกกรณี:
        - ถ้ามี 'messages': ใช้ messages เดิม (ปลอดภัย, สอดคล้อง chat template)
            * รูป: ค้นใน user.content[type=='image']
            * ref_caption: ใช้ assistant.content[type=='text'] ถ้าไม่มีคอลัมน์ 'text'
        - ถ้าไม่มี 'messages': ใช้ row['image'] + row['text'] แล้วสร้าง messages ใหม่

        คืนค่า: messages(dict), ref_caption(str), true_cls(str)
        """
        messages = None
        ref_caption = row.get("text", "") or ""       # เผื่อไม่มี 'text' จะอัปเดตจาก assistant ทีหลัง
        true_cls = row.get("__class__", "") or ""     # label จริงของคลาส

        if "messages" in row and isinstance(row["messages"], list):
            # ใช้ messages เดิมเพื่อเลี่ยง mismatch กับ processor/data_collator
            messages = {"messages": row["messages"]}

            # หา image จาก user turn แรกที่มี type=='image'
            image_found = False
            for turn in row["messages"]:
                if turn.get("role") == "user":
                    for c in (turn.get("content") or []):
                        if isinstance(c, dict) and c.get("type") == "image" and c.get("image") is not None:
                            image_found = True
                            break
                if image_found:
                    break

            # หา ref caption จาก assistant ถ้ายังไม่มี 'text'
            if not ref_caption:
                for turn in row["messages"]:
                    if turn.get("role") == "assistant":
                        for c in (turn.get("content") or []):
                            if isinstance(c, dict) and c.get("type") == "text" and c.get("text"):
                                ref_caption = c["text"]
                                break
                        if ref_caption:
                            break

            # ถ้า messages มีอยู่แต่ไม่มีภาพเลย (rare) → พยายาม fallback จากคีย์ 'image'
            if not image_found and row.get("image", None) is not None:
                messages = self.build_messages(row["image"])

        else:
            # โครงสร้างเก่า: ต้องมี 'image' จึงจะ build ได้
            img = row.get("image", None)
            if img is None:
                # ยังไม่มีภาพจริง ๆ → โยน error ที่เข้าใจง่ายแทน KeyError
                raise ValueError(
                    "No image found in row. Expected either 'messages' with an image content "
                    "or an 'image' column."
                )
            messages = self.build_messages(img)

        return messages, ref_caption, true_cls

    @torch.no_grad()
    def on_evaluate(self, args, state, control, model=None, **kwargs):
        model.eval()

        # ----- สุ่ม subset จาก val_ds เพื่อลดเวลา evaluate -----
        n = len(self.eval_dataset)
        idxs = list(range(n))
        self.rng.shuffle(idxs)
        idxs = idxs[:min(self.sample_size, n)]

        pred_classes, true_classes = [], []
        rouge_ls = []

        for i in idxs:
            row = self.eval_dataset[i]

            # ✅ ดึง messages/ref/label แบบกันพัง ไม่ผูกติดคีย์ 'image'
            messages, ref_caption, true_cls = self._extract_from_row(row)

            # ----- เตรียมอินพุตตาม template ของรุ่น (ใช้ messages ที่สกัดได้) -----
            inputs = self.tokenizer.apply_chat_template(
                messages["messages"],
                add_generation_prompt=True,
                tokenize=True,
                return_tensors="pt"
            ).to(model.device)

            # หมายเหตุ: Unsloth FastVisionModel จะ map ภาพจาก messages ผ่าน data_collator/processor ภายใน
            # ในกรณีรุ่นของคุณต้องการ kwargs เพิ่ม (เช่น pixel_values) ให้ปรับ data_collator ให้จ่ายมาพร้อมกัน

            # ----- Generate -----
            out = model.generate(
                input_ids=inputs,
                max_new_tokens=self.max_new_tokens,
                do_sample=False
            )
            text = self.tokenizer.decode(out[0], skip_special_tokens=True)

            # ----- แยก "คลาส" ที่โมเดลทำนาย พร้อมคำนวณ ROUGE-L caption -----
            pred_cls_str = extract_pred_class(text) or ""   # string ของคลาสที่พยากรณ์
            if pred_cls_str in LABEL_SET:
                pred_classes.append(LABEL_TO_ID[pred_cls_str])
            else:
                pred_classes.append(-1)  # ถ้าอ่านไม่ออกให้ mark -1 แล้วกรองภายหลัง

            true_classes.append(LABEL_TO_ID.get(true_cls, -1))

            # ถ้ายังไม่มี ref_caption จริง ๆ ให้ใส่ "" เพื่อให้ rouge_l_f1 คืน 0 แทนพัง
            rouge_ls.append(rouge_l_f1(text, ref_caption or ""))

        # ----- ทำความสะอาดกรณีมี -1 -----
        y_true_clean, y_pred_clean = [], []
        for yt, yp in zip(true_classes, pred_classes):
            if yt >= 0 and yp >= 0:
                y_true_clean.append(yt)
                y_pred_clean.append(yp)

        macro_f1 = macro_f1_from_predictions(y_true_clean, y_pred_clean) if y_true_clean else 0.0
        rougeL   = float(np.mean(rouge_ls)) if rouge_ls else 0.0

        # ----- บันทึกค่า metric เข้าสู่ระบบของ Trainer -----
        metrics = kwargs.get("metrics", {})
        metrics["macro_f1"] = macro_f1            # ✅ metric หลักสำหรับเลือก best
        metrics["rougeL"]   = rougeL              # ✅ proxy ของ caption คุณภาพ (0–1)
        kwargs["metrics"] = metrics

        return control


In [ ]:
# # ==========================
# # 4) Callback: Gate เมตริก captioning (ขั้นต่ำ) + Early stopping เสริม
# #    - ถ้า rougeL ต่ำกว่า min_value ติดต่อกัน 'patience' รอบ → หยุดเทรน
# #    - และไม่ให้ save checkpoint ในรอบที่ต่ำกว่ามาตรฐาน
# # ==========================
# class SecondaryMetricGate(TrainerCallback):
#     def __init__(self, metric_key="rougeL", min_value=0.35, patience=2):
#         self.metric_key = metric_key
#         self.min_value = min_value
#         self.patience = patience
#         self.bad_epochs = 0

#     def on_evaluate(self, args, state, control, **kwargs):
#         metrics = kwargs.get("metrics", {}) or {}
#         val = metrics.get(self.metric_key, None)
#         if val is None:
#             # ไม่มี metric caption รอบนี้ → ไม่ gate
#             return control

#         if val < self.min_value:
#             # 1) ไม่ save checkpoint รอบนี้ (กันเลือกเช็คพอยต์ที่ caption แย่มาก)
#             control.should_save = False
#             # 2) นับความล้มเหลวเพื่อ early stop
#             self.bad_epochs += 1
#             if self.bad_epochs >= self.patience:
#                 control.should_training_stop = True
#         else:
#             self.bad_epochs = 0

#         return control

In [38]:
# ==========================
# 4) Callback: Gate เมตริก captioning (ขั้นต่ำ) + Early stopping เสริม
#    - รองรับทั้งคีย์ "rougeL" และ "eval_rougeL" อัตโนมัติ
# ==========================
class SecondaryMetricGate(TrainerCallback):
    def __init__(self, metric_key="rougeL", min_value=0.35, patience=2):
        self.metric_key = metric_key          # เช่น "rougeL" หรือจะส่ง "eval_rougeL" มาก็ได้
        self.min_value = float(min_value)
        self.patience = int(patience)
        self.bad_epochs = 0

    def _get_metric(self, metrics: dict):
        """พยายามดึงค่าจากทั้งคีย์ปกติและคีย์ที่มี prefix 'eval_'"""
        if self.metric_key in metrics:
            return metrics[self.metric_key]
        # ถ้ายังไม่เจอ ลองเติม/ตัด prefix 'eval_'
        if self.metric_key.startswith("eval_"):
            alt = self.metric_key.replace("eval_", "", 1)
            return metrics.get(alt, None)
        else:
            alt = f"eval_{self.metric_key}"
            return metrics.get(alt, None)

    def on_evaluate(self, args, state, control, **kwargs):
        metrics = kwargs.get("metrics", {}) or {}
        val = self._get_metric(metrics)
        if val is None:
            # ไม่มี metric รอบนี้ → ไม่ gate
            return control

        if val < self.min_value:
            # 1) ไม่ save checkpoint รอบนี้ (กันเลือกเช็คพอยต์ที่ caption แย่มาก)
            control.should_save = False
            # 2) นับความล้มเหลวเพื่อตัดสินใจ early stop
            self.bad_epochs += 1
            if self.bad_epochs >= self.patience:
                control.should_training_stop = True
        else:
            self.bad_epochs = 0

        return control


In [39]:
# ==========================
# 5) เร่งคณิตศาสตร์บน GPU (TF32) + เปิด train
# ==========================
try:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

FastVisionModel.for_training(model)  # ✅ enable training
pass

In [40]:
# # ==========================
# # 6) สร้าง Trainer + ตั้งค่าให้เลือก "โมเดลดีที่สุด" ด้วย macro_f1
# #    - eval/save แบบ steps เพื่อให้ callback ทำงานถี่พอ
# #    - remove_unused_columns=False สำคัญกับ VLM (รักษา fields image/messages)
# # ==========================
# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     data_collator = UnslothVisionDataCollator(model, tokenizer),
#     train_dataset = train_ds,
#     eval_dataset = val_ds,
#     args = SFTConfig(
#         # ===== core =====
#         output_dir="./outs",
#         per_device_train_batch_size=2,
#         gradient_accumulation_steps=16,
#         learning_rate=1e-4,
#         num_train_epochs=2,
#         warmup_ratio=0.05,
#         weight_decay=0.01,
#         lr_scheduler_type="linear",

#         # ===== precision =====
#         bf16 = is_bf16_supported(),   # ✅ ใช้ตามฮาร์ดแวร์ (Ampere+)
#         tf32 = True,

#         # ===== eval/save =====
#         eval_strategy="steps",
#         eval_steps=300,
#         save_strategy="steps",
#         save_steps=300,
#         load_best_model_at_end=True,

#         # ✅ เลือกโมเดลที่ดีที่สุดตาม "macro_f1" (งานหลัก: classification)
#         metric_for_best_model="macro_f1",
#         greater_is_better=True,

#         logging_steps=10,
#         save_total_limit=3,           # ✅ กันพื้นที่พังจากเช็คพอยต์เยอะ

#         # ===== VLM safety =====
#         remove_unused_columns=False,  # ✅ อย่าตัดคอลัมน์ messages/image
#         dataloader_num_workers=0,     # ✅ กันปัญหา pickle/vision worker
#         dataset_num_proc=1,           # ✅ กัน map หลายโปรเซส (สเถียร)
#         per_device_eval_batch_size=2, # ✅ ลด VRAM ขณะ eval+generate
#     ),
#     # เรา "ไม่" ใช้ compute_metrics ที่ระดับ SFTTrainer ตรง ๆ
#     # เพราะ VLM+generate มักชนกับ _pad_across_processes; จึงย้ายไป callback
# )

In [41]:
# # ==========================
# # 7) ติดตั้ง Callback:
# #    - EarlyStopping: อิง metric_for_best_model (macro_f1)
# #    - CaptionEvalCallback: เติม macro_f1 + rougeL ลง metrics ทุกครั้งที่ eval
# #    - SecondaryMetricGate: กั้น checkpoint/หยุด หาก caption ต่ำกว่าเกณฑ์
# # ==========================
# trainer.add_callback(EarlyStoppingCallback(
#     early_stopping_patience=5,         # ถ้า macro_f1 ไม่ดีขึ้น 5 ครั้งติด → หยุด
#     early_stopping_threshold=1e-4
# ))
# trainer.add_callback(CaptionEvalCallback(
#     eval_dataset=val_ds,
#     tokenizer=tokenizer,
#     sample_size=64,                   # ✅ subset เพื่อความเร็ว (ปรับได้)
#     max_new_tokens=96
# ))
# trainer.add_callback(SecondaryMetricGate(
#     metric_key="rougeL",               # ✅ ใช้ ROUGE-L เป็น proxy ของ caption คุณภาพ
#     min_value=0.35,                    # ✅ เกณฑ์ขั้นต่ำ (ปรับตามฐาน)
#     patience=2
# ))

In [42]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer),
    train_dataset = train_ds,
    eval_dataset = val_ds,
    args = SFTConfig(
        output_dir="./outs_fast",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=1e-4,
        num_train_epochs=1,
        max_steps=120,
        warmup_ratio=0.0,
        weight_decay=0.0,
        lr_scheduler_type="linear",
        bf16 = is_bf16_supported(),
        tf32 = True,

        # ---- eval/save ----
        eval_strategy="steps",
        eval_steps=40,
        save_strategy="no",          # ไม่เซฟ เพื่อลด I/O
        load_best_model_at_end=False,# ไม่ต้องเลือก best

        # ⛔ ลบบรรทัดนี้ทิ้งในโหมดเทสต์:
        # metric_for_best_model="macro_f1",
        # greater_is_better=True,

        logging_steps=5,
        save_total_limit=1,

        remove_unused_columns=False,
        dataloader_num_workers=0,
        dataset_num_proc=1,
        per_device_eval_batch_size=1,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [43]:
# โหมดเร็ว: ใช้ subset เล็ก + generate สั้น
trainer.add_callback(CaptionEvalCallback(
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    sample_size=32,        # จาก 256 → 32 เร็วขึ้นมาก
    max_new_tokens=40      # จาก 96 → 40 พอจับสัญญาณได้
))
# เกทคุณภาพ caption ให้เบาลงหน่อย (ลดเวลาหยุดก่อนเวลา)
trainer.add_callback(SecondaryMetricGate(
    metric_key="rougeL",
    min_value=0.30,        # ผ่อนเกณฑ์ลงเล็กน้อยในโหมดเทสต์
    patience=1             # ถ้าต่ำ 1 รอบติดให้หยุด (ประหยัดเวลา)
))

In [44]:
# title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4050 Laptop GPU. Max memory = 5.997 GB.
6.238 GB of memory reserved.


In [ ]:
from unsloth import unsloth_train

trainer_stats = unsloth_train(trainer)